In [ ]:
%load_ext autoreload
%autoreload 2

## Load the model and set up a hook

In [167]:
import torch
from neuroai.models import ResNet18

backbone_model = ResNet18(pretrained=True, download_root="./pretrained_checkpoints")

## Extracting features

In [168]:
from neuroai.utils import ForwardHook

hook = ForwardHook(
    model=backbone_model, 
    hook_layer_name = "model.layer4.0.conv1" # << extract activations from this layer
)

In [ ]:
x = torch.randn(1,3,224,224)
y = backbone_model(x)

## it's of shape (batch, channels, height, width)
hook.output.shape

In [235]:
from neuroai.datasets import NSDAllSubjectSingleRegion
import torch.nn as nn

device = "mps:0"

## lets move the model to our device
backbone_model = backbone_model.to(device)

region = "PPA"
dataset = NSDAllSubjectSingleRegion(
    folder="./datasets/nsd",
    region=region,
    transforms=backbone_model.transforms,
    subset="train",
    train_test_split=0.8
)

In [236]:
def collect_features_and_voxels(
    dataset: NSDAllSubjectSingleRegion, 
    backbone_model: nn.Module, 
    hook: ForwardHook, 
    device: str, 
    subject_id: str
):
    
    ## this is where we dump our data
    all_features = []
    all_fmri_voxels = []

    with torch.no_grad():
        for i in range(len(dataset)):
            image: torch.Tensor = dataset[i]["image"]

            ## (voxels) -> (1, voxels)
            fmri_data: torch.Tensor = dataset[i]["fmri_response"][subject_id].unsqueeze(0)

            ## (channels, height, width) -> (1, channels, height, width)
            image = image.unsqueeze(0)
            image = image.to(device)
            y = backbone_model(image)

            ## making sure that we're moving stuff back to the RAM with .cpu()
            hook_output = hook.output.cpu()
            all_features.append(hook_output)
            all_fmri_voxels.append(fmri_data.cpu())

    return {
        "features": torch.cat(all_features, dim=0),
        "voxels": torch.cat(all_fmri_voxels, dim=0)
    }

In [237]:
subject_id = "s2"

data = collect_features_and_voxels(
    dataset=dataset, 
    backbone_model=backbone_model, 
    hook=hook, 
    device=device, 
    subject_id=subject_id
)

In [ ]:
## num samples, channels, height, width
print(f"Shape of features", data["features"].shape)

## num samples, voxels
print(f"Shape of voxels", data["voxels"].shape)

In [ ]:
from einops import rearrange

all_features_flattened = rearrange(
    data["features"],
    "batch channels height width -> batch (channels height width)"
)

print(f"Shape of features after flattening", all_features_flattened.shape)

In [240]:
from einops import reduce
all_fmri_voxels_region_mean = reduce(
    data["voxels"],
    pattern = "batch voxels -> batch",
    reduction="mean"
)

In [ ]:
from neuroai.utils.regression import ridge_regression

ridge_result = ridge_regression(
    X_train=all_features_flattened,
    Y_train=all_fmri_voxels_region_mean,
    device="cpu"
)
print(ridge_result)

In [249]:
from neuroai.utils.regression import RidgeModel

model = RidgeModel(
    backbone_model=backbone_model,
    transforms=backbone_model.transforms,
    hook_layer_name="model.layer4.0.conv1",
    ridge_result=ridge_result,
    device="mps:0"
)


## Now let's evaluate on data from another subject

In [ ]:
valid_subject_ids = ["s1", "s2", "s5", "s7"]

subject_id = "s1"

dataset = NSDAllSubjectSingleRegion(
    folder="./datasets/nsd",
    region=region,
    transforms=backbone_model.transforms,
    subset="test", ## << make sure that this is set to "test" when evaluating
    train_test_split=0.8
)

data = collect_features_and_voxels(
    dataset=dataset, 
    backbone_model=backbone_model, 
    hook=hook, 
    device=device, 
    subject_id=subject_id
)
all_features_flattened = rearrange(
    data["features"],
    "batch channels height width -> batch (channels height width)"
)
all_fmri_voxels_region_mean = reduce(
    data["voxels"],
    pattern = "batch voxels -> batch",
    reduction="mean"
)

print(all_features_flattened.shape, all_fmri_voxels_region_mean.shape)

In [ ]:
correlation = model.evaluate(
    x_test=all_features_flattened,
    y_test=all_fmri_voxels_region_mean,
)
print(f"Correlation on another subject {subject_id}: {correlation}")

## Running the model on your images

In [ ]:
# !wget -O cat.jpg "https://plus.unsplash.com/premium_photo-1667030474693-6d0632f97029"
# !wget -O castle.jpg "https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Panorámica_Otoño_Alcázar_de_Segovia.jpg/1200px-Panorámica_Otoño_Alcázar_de_Segovia.jpg"
# !wget -O monkey.jpg "https://images.unsplash.com/photo-1581828060707-37894f1ed9b8"

In [ ]:
images = [
        Image.open("monkey.jpg"),
        Image.open("castle.jpg")
    ]

results = model.run(
    images = images
)
print(results)
